<a href="https://colab.research.google.com/github/GlassesNoGlasses/TFProjects/blob/main/projects/text/Harry_Potter_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone from GitHub repository

!git clone https://github.com/GlassesNoGlasses/TFProjects.git

Cloning into 'TFProjects'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 0), reused 8 (delta 0), pack-reused 0
Receiving objects: 100% (14/14), 176.98 KiB | 4.21 MiB/s, done.


**Goal**: Generate text for a harry potter book. We will use RNN and Keras similar to the TensorFlow tutorial.

In [2]:
# Required Imports
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
# Obtain harry potter books in .txt form

#pathToFile = tf.keras.utils.get_file('harryPotterBook1.txt', 'file://content/TFProjects/data/texts/harryPotterBook1.txt')

text = open('/content/TFProjects/data/texts/harryPotterBook1.txt', 'rb').read().decode(encoding='utf-8')

In [4]:
# The unique characters in book 1

vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

79 unique characters


In [5]:
# Convert vocab into a list, then each character is tokenized with a unique id.

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [6]:
# Return characters based on their id representation defined above.

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [7]:
# Join ids back into original stirngs

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [8]:
# Tokenize and assign character ids to all characters in original text
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))


# Convert ids into a stream of ids that represent the original text characters
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [9]:
# Define the sequence length of characters to train model on
seq_length = 120

In [10]:
# Create sequential batches of size seq_length + 1
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

We are trying to predict the next character only.

In [11]:
# Split input sequence into a data set of (input, label)
# I.e. "tensorflow" = ("tensorflo", "ensorflow")

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [12]:
# Create training data set based on our original sequence
dataset = sequences.map(split_input_target)

In [13]:
# Creating test batches

BATCH_SIZE = 64

# Buffer to fit data into
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset


<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 120), dtype=tf.int64, name=None), TensorSpec(shape=(64, 120), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # vocab_size: unique inputs + 1
    # embedding_dim: output vector dimensions
    # rnn_units: how many rnn used.
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    # log liklihood with vocab_size outputs
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [16]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [17]:
# Loss function

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
# Configuration of model with optimizer and loss functions

model.compile(optimizer='adam', loss=loss)

In [19]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
# Actual Training process

EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
56/56 [==============================] - 10s 71ms/step - loss: 3.3672
Epoch 2/20
56/56 [==============================] - 4s 64ms/step - loss: 2.4739
Epoch 3/20
56/56 [==============================] - 4s 63ms/step - loss: 2.2376
Epoch 4/20
56/56 [==============================] - 5s 63ms/step - loss: 2.0590
Epoch 5/20
56/56 [==============================] - 4s 63ms/step - loss: 1.8972
Epoch 6/20
56/56 [==============================] - 4s 63ms/step - loss: 1.7597
Epoch 7/20
56/56 [==============================] - 4s 63ms/step - loss: 1.6437
Epoch 8/20
56/56 [==============================] - 4s 63ms/step - loss: 1.5452
Epoch 9/20
56/56 [==============================] - 5s 63ms/step - loss: 1.4651
Epoch 10/20
56/56 [==============================] - 4s 62ms/step - loss: 1.3969
Epoch 11/20
56/56 [==============================] - 4s 63ms/step - loss: 1.3394
Epoch 12/20
56/56 [==============================] - 5s 63ms/step - loss: 1.2854
Epoch 13/20
56/56 [=================

In [21]:
# Generating Text Class

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [22]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [24]:
start = time.time()
states = None
next_char = tf.constant(['CHAPTER'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

CHAPTER FIRTT

DIMAT OUREN

Harry felt as though the nearest had wagan clitting moutly to the living room bardling he was a terrible snaps.

"Mysnock," said Hagrid under she let the trail in the
edge."

"And what if Ron muttered," she said, his left exiced.

"See through the train," Hagrid whispered.

Who looked as for it into the hoise, long, looking larged telling himself.

"Mars ignor?"

"Why?" he noise and laughed.

"Snape!" Ron went off to
his feet, and
slithering voice an engule in castle -- they looked like a cholicat later.

Sme liking Fluffy from holding behind gloats. Did next her neck,
"Snape's also covered -- friends!" Hagrid pushed. "For once he'd just looked as it broh. He
pulled the des. Hagrid man his four heavy, grad their lead:

And long black bitch of Hagrid's blank to asky from 16ville's Slitwle
-- then morning, I mining warn you."

They were all tres. He got back at the
mad. Hagrid looked as though and thirteen that the porers didn't know
what was going to prick to

In [25]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [26]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

model.fit(dataset, epochs=1)

56/56 [==============================] - 10s 65ms/step - loss: 3.3734


In [30]:
EPOCHS = 50

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 0.8105
Epoch 1 Batch 50 Loss 0.8192

Epoch 1 Loss: 0.8189
Time taken for 1 epoch 10.24 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 0.7337
Epoch 2 Batch 50 Loss 0.8081

Epoch 2 Loss: 0.7629
Time taken for 1 epoch 5.11 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 0.6730
Epoch 3 Batch 50 Loss 0.7236

Epoch 3 Loss: 0.7059
Time taken for 1 epoch 4.05 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 0.6321
Epoch 4 Batch 50 Loss 0.6815

Epoch 4 Loss: 0.6490
Time taken for 1 epoch 4.09 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 0.5757
Epoch 5 Batch 50 Loss 0.6177

Epoch 5 Loss: 0.5898
Time taken for 1 epoch 4.21 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 0.5267
Epoch 6 Bat

In [35]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant(['CHAPTER 100'])
result = [next_char]

for n in range(1000):
  next_char, states =  one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

CHAPTER 100


INWARTS!" he shouted.

"Let me see it!" demanded Dudley.

"OUT!" roared Uncle Vernon that people laughed; and saw that Hagrid had gotten used to this by now, but
it had given him a bit of a shock on the first morning. He back down the
corner of strangely dressed people and the next morning when they
were all very impreces in the darkness.

Charlie's friends were a cheery lot. They showed Harry and Hermione the
harness they'd rate to do we to do. He shook his head and the
books strange and except for his abyea, could go to Dumbledore.
He hoped they were lost, warming toward the dungeon
ceiling.

"I don't know," said Harry quietly. "Ant me --"

A sudden noise outside in the corridor put an end to their discussion.
They hadn't realized how much never belonged him and he caught a
few moons when the compartment door slid open years.

The baby banged its tail on the wall, making the windows rattle. Harry
and Ron walked back to the castle for dinner, their pockets
weighed down w